In [1]:
import numpy as np
import pandas as pd

# Creating a MultiIndex (hierarchical index) object

In [2]:
arrays = [
    ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
    ["one", "two", "one", "two", "one", "two", "one", "two"],
]
arrays

[['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
 ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]

In [3]:
tuples = list(zip(*arrays))
print(tuples, type(tuples), sep='\n')

[('bar', 'one'), ('bar', 'two'), ('baz', 'one'), ('baz', 'two'), ('foo', 'one'), ('foo', 'two'), ('qux', 'one'), ('qux', 'two')]
<class 'list'>


In [4]:
index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])  # 从元组中创建MultiIndex
index

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('baz', 'one'),
            ('baz', 'two'),
            ('foo', 'one'),
            ('foo', 'two'),
            ('qux', 'one'),
            ('qux', 'two')],
           names=['first', 'second'])

In [5]:
s = pd.Series(np.random.randint(-10, 10, size=8), index=index)
s

first  second
bar    one       4
       two       4
baz    one      -5
       two      -1
foo    one       0
       two       3
qux    one       6
       two       8
dtype: int32

![结果](https://cdn.jsdelivr.net/gh/pumpbumb/pictures/img/20220407211913.png)

---

# Basic indexing on axis with MultiIndex

In [6]:
df = pd.DataFrame(np.random.randn(3, 8), index=["A", "B", "C"], columns=index)
df

first        bar                 baz                 foo                 qux  \
second       one       two       one       two       one       two       one   
A       0.290596 -0.623626 -1.451302 -0.701075  1.166159 -0.505487 -0.864885   
B       0.012239  0.116886 -0.308586 -0.031140 -0.352673  0.553499 -0.000723   
C      -0.774862  0.019877  0.841663 -1.551485 -0.209283 -0.969527 -0.619742   

first             
second       two  
A       0.316034  
B      -1.248362  
C       0.325351

In [7]:
df['bar']

second,one,two
A,0.290596,-0.623626
B,0.012239,0.116886
C,-0.774862,0.019877


In [8]:
df['bar', 'one']

A    0.290596
B    0.012239
C   -0.774862
Name: (bar, one), dtype: float64

In [9]:
df['bar']['one']

A    0.290596
B    0.012239
C   -0.774862
Name: one, dtype: float64

In [11]:
df.columns.levels

FrozenList([['bar', 'baz', 'foo', 'qux'], ['one', 'two']])

In [13]:
df[["foo", "qux"]].columns.get_level_values(0)

Index(['foo', 'foo', 'qux', 'qux'], dtype='object', name='first')

In [14]:
df[["foo", "qux"]].columns.get_level_values(1)

Index(['one', 'two', 'one', 'two'], dtype='object', name='second')

# Data alignment and using `reindex`

Operations between differently-indexed objects having MultiIndex on the axes will work as you expect; data alignment will work the same as an Index of tuples: